# DASC 5300, Fall2024, University of Texas at Arlington
# PA4 "Implementing the Go Fish Card Game in Python"


## **Academic Honesty**
This assignment must be done individually and independently. You must implement the whole assignment by yourself. Academic dishonesty is not tolerated.

## **Requirements**

1.   When you work on this assignment, you should make a copy of this notebook in Google Colab. This can be done using the option `File > Save a copy in Drive` in Google Colab.

2.  To submit your assignment, download your Colab into a .ipynb file. This
can be done using the option `Download > Download .ipynb` in Google Colab. Submit the downloaded .ipynb file/ .zip into the PA4 entry in Canvas.


## **Description**

Implementing the Go Fish card game using stacks and queues in Python. The program should simulate a game between two players, where each player draws cards from a central deck to collect sets of matching cards. The game ends when the deck is empty, and the player with the most sets at the end of the game wins. The program should use QUEUE to represent each player's hand and a STACK to represent the central deck of cards. The game should be playable in the command-line interface, with the option to specify the number of players and the number of cards dealt to each player at the start of the game."

In [10]:
import random
from collections import deque

class GoFishGame:
    def __init__(self, num_players, initial_cards, player_names):
        self.main_deck = self.create_deck()                         # The main deck is the central deck of cards
        self.hands = [deque() for _ in range(num_players)]          # List of deques for each player
        self.books = [0] * num_players                              # To keep record of books for each player
        self.players = player_names
        self.num_players = num_players                              # No. of players playing (can be 2 or more)
        self.initial_cards = initial_cards                          # No. of cards to be initially distributed (5-7)
        self.deal_initial_cards()

    def create_deck(self):
        suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
        values = [str(x) for x in range(2, 11)] + ['J', 'Q', 'K', 'A']
        deck = [value for value in values for _ in suits]                 # making deck of 4 suits and 13 values
        random.shuffle(deck)                                              # random suffling of deck
        return deck

    def deal_initial_cards(self):
        total_cards_needed = self.num_players * self.initial_cards
        if total_cards_needed > len(self.main_deck):                                 # total cards distributed should be <= cards in deck
            print("\nNot enough cards in the deck to deal the requested number!")
            print(f"The main deck only contains {len(self.main_deck)} cards.")
            while True:
                try:
                    self.initial_cards = int(input(f"Enter a smaller number of cards to deal (5 to {len(self.main_deck) // self.num_players}): "))
                    if 5 <= self.initial_cards <= len(self.main_deck) // self.num_players:
                        break
                    print("Please enter a valid number.")
                except ValueError:
                    print("Invalid input. Please enter an integer.")

        for _ in range(self.initial_cards):                              # The loop will run once for each card to be dealt to every player
            for hand in self.hands:                                      # The loop ensures that each player gets one card in every iteration of upper loop.
                if self.main_deck:                                       # This checks whether self.main_deck is not empty, it must contain cards for the player to draw from it.
                    hand.append(self.main_deck.pop())

    def draw_card(self, player_id):
        if self.main_deck:                                               # This checks if the deck is not empty.
            card = self.main_deck.pop()                                  # It pops a card from the main deck and gives it to the player.
            self.hands[player_id].append(card)
            print(f"{self.players[player_id]} draws a card: {card}.")
        else:
            print("The main deck is empty!")

    def check_books(self, player_id):
        hand = self.hands[player_id]
        counts = {}
        for card in hand:                                               # iterates through the player's hand, counting how many of each card they have using the counts dictionary.
            counts[card] = counts.get(card, 0) + 1                      # Retrieves the current count of card (or 0 if the card is not in counts) and adds 1 to it.

        books = [card for card, count in counts.items() if count == 4]
        for book in books:
            self.books[player_id] += 1                                               # This line increments the count of books for the current player by 1, showing the newly formed book.
            self.hands[player_id] = deque(card for card in hand if card != book)     # This updates the player's hand by removing all cards that were part of the book.
            print(f"{self.players[player_id]} forms a book with: {book}.")

    def take_turn(self, current):
        if not self.hands[current]:                                                  # If player has no cards in hand then has to draw 1 from main deck
            print(f"{self.players[current]} has no cards and must draw.")
            self.draw_card(current)
            return

        print(f"{self.players[current]}'s hand: {list(self.hands[current])}")        # Display current player's hand

        # Ask for a card
        while True:
            card_to_ask = input(f"{self.players[current]}, which card do you want to ask for? ").strip()
            if card_to_ask in self.hands[current]:
                break
            print("You can only ask for cards in your hand. Try again.")

        print("\nChoose an opponent:")                                               # Display the numbered list of opponents
        for i, name in enumerate(self.players):
            if i != current:
                print(f"{i}: {name}")

        # Choose an opponent by number
        while True:
            try:
                opponent = int(input(f"\n{self.players[current]}, select an opponent by number: "))
                if opponent != current and 0 <= opponent < self.num_players:
                    break
                print("Invalid choice. Please select a valid opponent number.")
            except ValueError:
                print("Invalid input. Please enter a number.")

        print(f"{self.players[current]} asks {self.players[opponent]} for: {card_to_ask}.")

        # Check if the opponent has the requested card #
        if card_to_ask in self.hands[opponent]:
            print(f"{self.players[opponent]} has the card(s)!")
            # Transfer all matching cards
            matching_cards = [card for card in self.hands[opponent] if card == card_to_ask]
            for card in matching_cards:
                self.hands[current].append(card)
                self.hands[opponent].remove(card)
            print(f"{self.players[current]} takes {len(matching_cards)} '{card_to_ask}' from {self.players[opponent]}.")

        # if not, the player has to "Go Fish" #
        else:
            print(f"{self.players[opponent]} does not have the card. Go Fish!")
            self.draw_card(current)

    def play_game(self):
        print("Starting Go Fish!")
        current = 0
        round_count = 0
        while self.main_deck:                                                         # Loop will run until cards remaining in main deck
            round_count += 1
            print("\n-------------------[Round count: {}]-------------------".format(round_count))
            print(f"main deck count: {len(self.main_deck)}")                          # Show center deck count
            print(f"\n{self.players[current]}'s turn.")
            self.take_turn(current)
            self.check_books(current)

            current = (current + 1) % self.num_players                                # The current player is incremented with the turn alternates between players.

        print("\nGame Over!")
        for i, books in enumerate(self.books):
            print(f"{self.players[i]} formed {books} book(s).")

        max_books = max(self.books)
        winners = [self.players[i] for i, count in enumerate(self.books) if count == max_books]
        if len(winners) == 1:
            print(f"\n{winners[0]} wins with {max_books} book(s)!")
        else:
            print(f"\nIt's a tie between: {', '.join(winners)}!")

if __name__ == "__main__":
    # Asking user for the number of players
    while True:
        try:
            num_players = int(input("Enter the number of players (2 to 10): "))
            if num_players >= 2 and num_players <= 10:
                break
            print("The game requires at least 2 players and atmost 10 players.")
        except ValueError:
            print("Invalid input. Please enter an integer.")

    # Asking for player names
    player_names = []
    for i in range(num_players):
        name = input(f"Enter the name for Player {i + 1}: ")
        player_names.append(name)

    # Asking for no. of initial cards to distribute
    while True:
        try:
            initial_cards = int(input(f"Enter the number of cards to distribute (5 to 7): "))
            if 5 <= initial_cards <= 7:
                break
            print("Please enter a number between 5 and 7.")
        except ValueError:
            print("Invalid input. Please enter an integer.")

    # the game starts!!
    GoFishGame(num_players=num_players, initial_cards=initial_cards, player_names=player_names).play_game()

# Submitted by Anuva Negi

Enter the number of players (2 to 10): 2
Enter the name for Player 1: Anuva
Enter the name for Player 2: Adit
Enter the number of cards to distribute (5 to 7): 7
Starting Go Fish!

-------------------[Round count: 1]-------------------
main deck count: 38

Anuva's turn.
Anuva's hand: ['9', '10', 'Q', '5', '3', '7', 'A']
Anuva, which card do you want to ask for? 5

Choose an opponent:
1: Adit

Anuva, select an opponent by number: 1
Anuva asks Adit for: 5.
Adit does not have the card. Go Fish!
Anuva draws a card: 3.

-------------------[Round count: 2]-------------------
main deck count: 37

Adit's turn.
Adit's hand: ['J', 'J', '9', '2', '10', 'A', 'K']
Adit, which card do you want to ask for? J

Choose an opponent:
0: Anuva

Adit, select an opponent by number: 0
Adit asks Anuva for: J.
Anuva does not have the card. Go Fish!
Adit draws a card: 10.

-------------------[Round count: 3]-------------------
main deck count: 36

Anuva's turn.
Anuva's hand: ['9', '10', 'Q', '5', '3', '7', 'A', 